In [1]:
import pandas as pd
import numpy as np
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

d:\programas\programas\github repos\nlp\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/preprocessed_train.csv', encoding='utf-8')
df.head()

,title,author,text,label,removed_punc,tokens,filtered_tokens,clean_tokens,lemma_words,clean_text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide We Didn’t Even See Comey’s Lett...,"['house', 'dem', 'aide', 'we', 'didn’t', 'even...","['house', 'aide', 'didn’t', 'even', 'comey’s',...","['house', 'aide', 'didn’t', 'even', 'comey’s',...","['house', 'aide', 'didn’t', 'even', 'comey’s',...",house aide didn’t even comey’s letter jason ch...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,Ever get the feeling your life circles the rou...,"['ever', 'get', 'the', 'feeling', 'your', 'lif...","['ever', 'feeling', 'your', 'life', 'circles',...","['ever', 'feeling', 'life', 'circles', 'rounda...","['ever', 'feeling', 'life', 'circle', 'roundab...",ever feeling life circle roundabout rather hea...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired October 29 2...,"['why', 'the', 'truth', 'might', 'get', 'you',...","['truth', 'might', 'fired', 'october', '2016',...","['truth', 'might', 'fired', 'october', '2016',...","['truth', 'might', 'fired', 'october', '2016',...",truth might fired october 2016 tension intelli...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Videos 15 Civilians Killed In Single US Airstr...,"['videos', '15', 'civilians', 'killed', 'in', ...","['videos', 'civilians', 'killed', 'single', 'a...","['videos', 'civilians', 'killed', 'single', 'a...","['video', 'civilian', 'killed', 'single', 'air...",video civilian killed single airstrike identif...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Print \nAn Iranian woman has been sentenced to...,"['print', 'an', 'iranian', 'woman', 'has', 'be...","['print', 'iranian', 'woman', 'been', 'sentenc...","['print', 'iranian', 'woman', 'sentenced', 'ye...","['print', 'iranian', 'woman', 'sentenced', 'ye...",print iranian woman sentenced year prison iran...


In [3]:
data = df.loc[df['label'] == 1, 'clean_text']
text_list = data.tolist()
text_list= [text for text in text_list if not (isinstance(text, float) and np.isnan(text))]
#del text_list[7406] #este indice es un string con palabras en ruso


# Generator with BERT

### Tokenization

In [4]:
from transformers import BertTokenizer, BertForMaskedLM, AdamW

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Tokenize your dataset
max_length = 512
tokenized_dataset = tokenizer(text_list, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

# Get the input IDs and attention mask tensors from the tokenized dataset
input_ids = tokenized_dataset['input_ids']
attention_mask = tokenized_dataset['attention_mask']

# Define the optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=5e-5)
# Calculate the length of each input sequence
input_lengths = [len(input_seq) for input_seq in input_ids]

d:\programas\programas\github repos\nlp\nlp\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Batch training

In [6]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_dataset):
        self.input_ids = tokenized_dataset['input_ids']
        self.attention_mask = tokenized_dataset['attention_mask']

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx]

    def __len__(self):
        return len(self.input_ids)

In [7]:
dataset = TextDataset(tokenized_dataset)

# Define your data loader
batch_size = 8
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

### Model training

In [8]:
# Define your training loop
num_epochs = 5
for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids = batch[0]
        attention_mask = batch[1]

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f'Epoch {epoch+1} loss: {loss.item()}')

Epoch 1 loss: 0.43760988116264343
Epoch 2 loss: 0.1546461433172226
Epoch 3 loss: 0.11059565842151642


### Model Output

In [10]:
# Define the prompt
prompt = "The president will"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt')

# Generate text
max_length = 50
output = model.generate(input_ids=input_ids, max_length=max_length, do_sample=True, repetition_penalty=2.0)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

the president will ’ [unused615] viet final residual conclusion niall conclusions ” suspension “ quentin discovery patrice lethal chloe thanksgivinghairrocklmanwee elimination nrl fran bryn horribly niall countless colbertoganlai niall steam niallree rwandabh genocide chemical resultlay conan vampire commencement bismarck civilian


### Model configuration

### Model Training / fine tuning

### Model Output

# Generator with Transformers

In [53]:
text_data = text_list[:6000]

### tokenization of data

In [13]:
# Tokenize the dataset
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenized_dataset = tokenizer.encode(text_data, add_special_tokens=True, max_length=1024, truncation=True)

# Convert the tokenized dataset to tensors
input_ids = torch.tensor(tokenized_dataset[:-1]).unsqueeze(0)
labels = torch.tensor(tokenized_dataset[1:]).unsqueeze(0)

dataset = torch.utils.data.TensorDataset(input_ids, labels)

# Create data loaders
train_loader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Fine-tune the model
optimizer = AdamW(model.parameters(), lr=1e-5)

model.train()

d:\programas\programas\github repos\nlp\nlp\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

### Model Training

In [14]:
for epoch in range(5):
    for batch in train_loader:

        # Get the input and target sequences
        input_ids = batch[0]
        target_ids = batch[1]

        # Forward pass
        outputs = model(input_ids, labels=target_ids)

        # Compute the loss
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Zero the gradients
        optimizer.zero_grad()



### Model Output

Si se juega con los parametros ingresados en el metodo generate es posible cambiar la estructura de la generación de noticias. Revisar la documentación.

In [24]:
seed_text = "Terrorists"
input_ids = tokenizer.encode(seed_text, return_tensors='pt')
output = model.generate(
    input_ids,
    max_length=300,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=2.0,
)
# Decode the generated sequences
generated_text = []
for sequence in output:
    sequence = sequence.tolist()
    text = tokenizer.decode(sequence, skip_special_tokens=True)
    generated_text.append(text)

print("Generated Fake News: ", generated_text)

Generated Fake News:  ['Terrorists.\nThe US has been a major supporter of the Syrian regime and its allies, including Iran, have long supported Assad\'s government in Syria as well. The Obama administration is now trying to get it into power by using sanctions on Russia for supporting Bashar al-Assad while also backing his ally Turkey which supports him with military aid from Moscow (see "Russia Is Supporting ISIS" here). This will be an extremely difficult situation if there are any Russian support or even close ties between these two countries that would make them very hard targets at this point because they both want their own country back against Putin who wants control over all aspects within our borders so he can use us without being able access through other nations\' territory where we don\'t need help but what I\'m saying right? We\'re not going anywhere near anything like those things happening today when you look around your world just looking out across space."']


### Perplexity testing

In [42]:
test = text_list[6000:]
test

['anunnaki’s son given push development minor god alone universe exists entity advanced entity watching visiting periodically planet religion certain teach basically need reformulated case calling higher entity much engraved thousand tablet survived 5000 year preserved best possible burnt clay clay tablet cooked fire library nineveh abnormal fact given extra resistance endure time question anunnaki’s real universal real universal much abstract togetherness universal consciousness holly spirit togetherness consciousness universal abstract idea people tended forget revert humanized type god quantum theory tell creating world resides source free conscience influence wave collapse everything wave mean part universal anyone influence everyone evil man’s creation military economic cartel kept important discovery particularly field energy generation hidden year doubt delicate important subject read book steven greer published called “disclosure” steven doctor working emergency room american h

In [51]:
test_set_tokens = tokenizer.encode(test, add_special_tokens=True, max_length=1024, truncation=True)
# Convert the tokenized dataset to tensors
input_ids = torch.tensor(test_set_tokens[:-1]).unsqueeze(0)
labels = torch.tensor(test_set_tokens[1:]).unsqueeze(0)

In [52]:
loss = model(input_ids = input_ids, labels = labels).loss
ppl = torch.exp(loss)
print("Perplexity score: ", ppl)

Perplexity score:  tensor(1.4731, grad_fn=<ExpBackward0>)
